### Este Notebook esta creado para la descarga de los datos, procesamiento y limpieza

##### 0. Carga de librerias

In [2]:

import pandas as pd
import numpy as np
import re
import os
import requests
import unidecode


##### 1. Descarga de datos abiertos

In [3]:
# Dirección donde se contiene "Quíenn es quien"
# URL base donde están los archivos
import os
import requests

# URL base donde están los archivos
base_url = "https://datos.profeco.gob.mx/datos_abiertos/"

# Crear la carpeta 'Data' si no existe
data_path = '../Data'
if not os.path.exists(data_path):
    os.makedirs(data_path)

# Encabezados adicionales que pueden ser necesarios para la descarga
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Referer': 'https://datos.profeco.gob.mx/datos_abiertos/qqp.php'
}

# Diccionario de años y sus respectivos tokens de descarga
files = {
    "2018": "01fafa951fb6c82e6e4bb491af8f1688",
    "2019": "09939d92d2afcde64dbc06e057877e16",
    "2020": "4df382eefa26f1f0d28d3a11aaf41add",
    "2021": "493b83b886f0266909d783fc8f776b11",
    "2022": "af88f42c5cb82c6c35dd962b1ae69051",
    "2023": "f3270d227f2966e6138a3ed41a9bbfb7",
    "2024": "d96db96e920de49718814ea5715f6cd9"
}

# Descargar los archivos y guardarlos en la carpeta 'Data'
for year, token in files.items():
    # Formar la URL de descarga
    download_url = f"{base_url}file.php?t={token}"
    
    # Descargar el archivo con stream para manejar archivos grandes
    response = requests.get(download_url, headers=headers, stream=True)
    if response.status_code == 200:
        rar_path = os.path.join(data_path, f"QQP_{year}.rar")
        
        # Guardar el archivo RAR descargado
        with open(rar_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"El archivo para el año {year} ha sido descargado correctamente en {rar_path}.")
    else:
        print(f"Error al descargar el archivo para el año {year}: Status code {response.status_code}")



El archivo para el año 2018 ha sido descargado correctamente en ../Data\QQP_2018.rar.
El archivo para el año 2019 ha sido descargado correctamente en ../Data\QQP_2019.rar.
El archivo para el año 2020 ha sido descargado correctamente en ../Data\QQP_2020.rar.
El archivo para el año 2021 ha sido descargado correctamente en ../Data\QQP_2021.rar.
El archivo para el año 2022 ha sido descargado correctamente en ../Data\QQP_2022.rar.
El archivo para el año 2023 ha sido descargado correctamente en ../Data\QQP_2023.rar.
El archivo para el año 2024 ha sido descargado correctamente en ../Data\QQP_2024.rar.


##### 2. ¡Importante! extrae manualmente la información contenida en los archivos .rar a su carpeta del año correspondiente: 2018, 2019, ..., 2014. Dentro de la carpeta Datos

##### 3. Procesamiento y limpieza de los archivos csv, enfocados en la sección de electrodoméstios

In [6]:
#Una vez que en la carpeta Data, se encuentran las carpetas correspondientes a los años 2018 al 2024 con los csv contenidos en ella procede a correr la siguiente celda

def remove_accents(input_str):
    # Elimina los acentos de un string
    return unidecode.unidecode(input_str)

def clean_text(text):
    # Convierte a minúsculas, quita acentos y elimina caracteres no alfanuméricos excepto espacios
    text = remove_accents(text.lower())
    text = re.sub(r'[^a-z0-9\s]', '', text)  # Conserva los espacios inicialmente
    text = re.sub(r'\s+', '_', text)  # Reemplaza uno o más espacios por guión bajo
    return text

def clean_address(address):
    # Reemplaza comas por punto y coma en una dirección
    if not isinstance(address, str):
        return address
    return re.sub(r",", ";", address)

def clean_csv(file_path, output_path):
    # Lee un archivo CSV, limpia su contenido y lo guarda en una nueva ubicación
    try:
        data = pd.read_csv(file_path, dtype=str, on_bad_lines='skip')
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return

    # Define los nombres de las columnas
    columns = [
        'producto',
        'presentacion',
        'marca',
        'categoria',
        'catalogo',
        'precio',
        'fecha_registro',
        'cadena_comercial',
        'giro',
        'nombre_comercial',
        'direccion',
        'estado',
        'municipio',
        'latitud',
        'longitud'
    ]
    # Comprueba que el número de columnas sea correcto y renombra las columnas
    if len(data.columns) == len(columns):
        data.columns = columns
    else:
        print(f"Column mismatch in {file_path}, expected {len(columns)} but got {len(data.columns)}")
        return

    # Limpia la dirección
    data['direccion'] = data['direccion'].apply(clean_address)

    # Aplica la función de limpieza de texto a las columnas especificadas
    text_columns = ['producto', 'presentacion', 'marca', 'categoria', 'catalogo', 'cadena_comercial', 'giro', 'nombre_comercial', 'direccion', 'estado', 'municipio']
    for col in text_columns:
        data[col] = data[col].apply(lambda x: clean_text(x) if isinstance(x, str) else x)

    # Guarda los datos limpios en un nuevo archivo CSV
    data.to_csv(output_path, sep='|', index=False, encoding='utf-8')
    print(f"Archivo {output_path} limpiado y guardado exitosamente.")

# Directorio raíz donde se almacenan los archivos por año
root_dir = '../Data'
for year in range(2018, 2025):
    year_folder = os.path.join(root_dir, str(year))
    if os.path.exists(year_folder):
        # Procesa cada archivo CSV en la carpeta del año correspondiente
        for file_name in os.listdir(year_folder):
            if file_name.endswith('.csv') and not file_name.startswith('clean_'):
                file_path = os.path.join(year_folder, file_name)
                output_path = os.path.join(year_folder, f"clean_{file_name}")
                clean_csv(file_path, output_path)



Archivo ../Data\2018\clean_012018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_022018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_032018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_042018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_052018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_062018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_072018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_082018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_092018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_102018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_112018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_122018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_132018.csv limpiado y guardado exitosamente.
Archivo ../Data\2018\clean_142018.csv limpiado y guardado exitos

OSError: [Errno 28] No space left on device

##### 4. Elimina los archivos raw .csv y .rar que no sean de utilidad para los fines correspondientes a los objetivos

In [7]:
# Ruta a la carpeta 'Data'
data_path = '../Data'

# Eliminar archivos .rar en la carpeta 'Data'
for item in os.listdir(data_path):
    if item.endswith('.rar'):
        os.remove(os.path.join(data_path, item))
        print(f"Eliminado: {item}")

# Años para los que se necesita limpiar archivos CSV
years = range(2018, 2025)

# Eliminar archivos CSV que no contienen 'clean_' en su nombre en las subcarpetas de cada año
for year in years:
    year_folder = os.path.join(data_path, str(year))
    if os.path.exists(year_folder):
        for file in os.listdir(year_folder):
            if file.endswith('.csv') and 'clean_' not in file:
                os.remove(os.path.join(year_folder, file))
                print(f"Eliminado: {file} en {year_folder}")


Eliminado: QQP_2018.rar
Eliminado: QQP_2019.rar
Eliminado: QQP_2020.rar
Eliminado: QQP_2021.rar
Eliminado: QQP_2022.rar
Eliminado: QQP_2023.rar
Eliminado: QQP_2024.rar
Eliminado: 012018.csv en ../Data\2018
Eliminado: 022018.csv en ../Data\2018
Eliminado: 032018.csv en ../Data\2018
Eliminado: 042018.csv en ../Data\2018
Eliminado: 052018.csv en ../Data\2018
Eliminado: 062018.csv en ../Data\2018
Eliminado: 072018.csv en ../Data\2018
Eliminado: 082018.csv en ../Data\2018
Eliminado: 092018.csv en ../Data\2018
Eliminado: 102018.csv en ../Data\2018
Eliminado: 112018.csv en ../Data\2018
Eliminado: 122018.csv en ../Data\2018
Eliminado: 132018.csv en ../Data\2018
Eliminado: 142018.csv en ../Data\2018
Eliminado: 152018.csv en ../Data\2018
Eliminado: 162018.csv en ../Data\2018
Eliminado: 172018.csv en ../Data\2018
Eliminado: 182018.csv en ../Data\2018
Eliminado: 192018.csv en ../Data\2018
Eliminado: 202018.csv en ../Data\2018
Eliminado: 212018.csv en ../Data\2018
Eliminado: 222018.csv en ../Data\2